In [1]:
import pandas as pd
import numpy as np
import mysql.connector as connection
import random
from gensim.test.utils import common_texts
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

'''
def make_doc2vec_models
save 경로 변경해주세요.
'C:/Users/SUN/바탕 화면/sesac/project/NEWS_TEAM_5/Data/doc2vec/' 이 부분 변경해주시면 됩니다.
'''
def make_doc2vec_models(tagged_data, tok, vector_size=128, window=3, epochs=40, min_count=0, workers=4):
    model = Doc2Vec(tagged_data, vector_size=vector_size, window=window, epochs=epochs, min_count=min_count, workers=workers)
    model.save(f"C:/Users/SUN/바탕 화면/sesac/project/NEWS_TEAM_5/Data/doc2vec/{tok}_news_model.doc2vec")



'''
def get_data
'database ='
'query ='에서 db이름 변경해주세요.
'''
def get_data(preprocess = True):
    if preprocess:
        try:
            database = connection.connect(host="localhost", database = 'newsdb', user="root", passwd="1111", use_pure=True)
            query = "SELECT * FROM newsdb.tb_news where DomainID = 1 and MainCategory = '생활/문화';"
            data = pd.read_sql(query,database)
            database.close()
        except Exception as e:
            database.close()
            print(str(e))
    
    return data



def get_preprocessing_data(data):
    data.drop(['WritedAt', 'URL', 'PhotoURL', 'Writer', 'Stickers', 'Press'], axis=1, inplace=True)
    
    # data['MainCategory'] = data['MainCategory']
    # data['SubCategory'] = data['SubCategory']
    data['Title_Content'] = data['Title'] + ' ' + data['Content']
    data.drop(['Title', 'Content'], axis = 1, inplace = True)
    return data



def make_doc2vec_data(data, column, t_document=False):
    
    data_doc = []

    for tag, doc in zip(data.index, data[column]):
        doc = doc.split(" ")
        data_doc.append(([tag], doc))
    if t_document:
        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
        return data
    else:
        return data_doc



def get_recommend_contents(user, data_doc, model):
    
    scores = []

    for tags, text in data_doc:
        trainded_doc_vec = model.docvecs[tags[0]]
        scores.append(cosine_similarity(user.reshape(-1,128), trainded_doc_vec.reshape(-1,128)))

    scores = np.array(scores).reshape(-1)
    scores = np.argsort(-scores)[:10] # 10개 추천

    return data.loc[scores, :]



def make_user_embedding(index_list, data_doc, model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.docvecs[i])
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding, axis = 0)
    return user



def view_user_history(data):
    print(data[['MainCategory', 'SubCategory', 'Title_Content']])

In [2]:
# 데이터 불러오기
data = get_data()

In [3]:
# column drop
# Title + Content 한 컬럼으로 만들어서 출력
get_preprocessing_data(data)

,ID,DomainID,MainCategory,SubCategory,Title_Content
0,226154,1,생활/문화,여행/레저,"괴산 은행나무길과 문광저수지, 늦가을 여행지로 안성맞춤 기자충북 괴산군 문광면..."
1,226155,1,생활/문화,여행/레저,"괌, 최근 1년간 관광객수 20만명 돌파…전년 대비 252%↑ 괌정부관광청은 202..."
2,226156,1,생활/문화,여행/레저,깊어가는 수승대의 가을 서울 1일 경남 거창군 위천면 수승대에 오색 단풍으로 가을이...
3,226157,1,생활/문화,여행/레저,깊어가는 수승대의 가을 서울 1일 경남 거창군 위천면 수승대에 오색 단풍으로 가을이...
4,226158,1,생활/문화,여행/레저,깊어가는 수승대의 가을 서울 1일 경남 거창군 위천면 수승대에 오색 단풍으로 가을이...
...,...,...,...,...,...
16980,243141,1,생활/문화,패션/뷰티,가상과 현실을 오가는 메타패션 쇼케이스 서울 기자 30일 서울 강남구 삼성동 섬...
16981,243142,1,생활/문화,패션/뷰티,메타패션 런웨이 서울 기자 30일 서울 강남구 삼성동 섬유센터에서 열린 메타패션...
16982,243143,1,생활/문화,패션/뷰티,"자라, 아더에러와 두번째 협업 프로젝트…성수동서 팝업 전시 서울 기자 스페인 패..."
16983,243144,1,생활/문화,패션/뷰티,"삼성물산 패션, SFDF 수상자에 ‘애슐린’ 박상연 디자이너 선정 삼성물산 패션부문..."


In [4]:
# data tagging
data_doc_title_content_tag = make_doc2vec_data(data, 'Title_Content', t_document=True)

In [5]:
data_doc_title_content = make_doc2vec_data(data, 'Title_Content')

In [6]:
# make doc2vec models
make_doc2vec_models(data_doc_title_content_tag, tok=False)

In [7]:
# load doc2vec models
'''
load 경로 변경
위에 def make_doc2vec_models 에서 설정한 경로로 수정해주세요.
'''
model_title_content = Doc2Vec.load('C:/Users/SUN/바탕 화면/sesac/project/NEWS_TEAM_5/Data/doc2vec/False_news_model.doc2vec')

In [11]:
# 유저 카테고리 정의
# 전체 생활/문화 데이터 중 랜덤으로 20개 추출
user_category = data.loc[random.sample(data.loc[data.MainCategory == '생활/문화', :].index.values.tolist(), 20), :]  # 생활/문화
view_user_history(user_category)

      MainCategory SubCategory  \
16950        생활/문화          날씨   
16042        생활/문화    생활/문화 일반   
11022        생활/문화        건강정보   
16876        생활/문화          날씨   
15685        생활/문화        건강정보   
11731        생활/문화       공연/전시   
149          생활/문화     자동차/시승기   
11020        생활/문화        건강정보   
5783         생활/문화     자동차/시승기   
4443         생활/문화       공연/전시   
5599         생활/문화          날씨   
13287        생활/문화    생활/문화 일반   
6060         생활/문화    생활/문화 일반   
6476         생활/문화          날씨   
3557         생활/문화       여행/레저   
15632        생활/문화     자동차/시승기   
5313         생활/문화    생활/문화 일반   
8031         생활/문화    생활/문화 일반   
3487         생활/문화          날씨   
13579        생활/문화       패션/뷰티   

                                           Title_Content  
16950  청주서 한파경보 속 첫눈 관측…작년보다 8일 늦어 청주  기자  11월의 마지막 날...  
16042  천혜의 아름다움, 서귀포의 자연…제주서 '특별전시회'   기자 천혜의 아름다움을 자...  
11022  '잊혀진 비타민'?...과다 결핍 땐 편두통, 발작 등 다만 이번 연구의 규모가 작...  
16876  무등산에 내린 올해 첫 눈 광주  기자  광주전남 최저기온이 영하권으로 떨어

In [12]:
# user_history 결과 바탕으로 embedding
user = make_user_embedding(user_category.index.values.tolist(), data_doc_title_content, model_title_content) # 생활/문화

c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\ipykernel_launcher.py:88: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).


In [13]:
# 컨텐츠 추천(함수에서 정의한대로 10개 출력)
result = get_recommend_contents(user, data_doc_title_content, model_title_content)
pd.DataFrame(result.loc[:, ['MainCategory', 'SubCategory', 'Title_Content']])

c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `docvecs` (The 

,MainCategory,SubCategory,Title_Content
14317,생활/문화,생활/문화 일반,"내방가사, 세계기록유산 아태 지역목록 등재 서울 문화재청은 올해 6월 등재 신청한 ..."
14747,생활/문화,날씨,"[포토]서울날씨 ""-1도 영하권추위"" 기자 중부 내륙과 전북 내륙 경북 북부에 ..."
2865,생활/문화,날씨,가을 날씨 즐기는 시민들 서울 기자 완연한 가을 날씨를 보인 6일 오전 서울 중...
4673,생활/문화,생활/문화 일반,'장영실' 국립현대미술관 과천 '백남준효과' 기획전 과천 기자 비디오 아트 선구자...
14315,생활/문화,생활/문화 일반,"내방가사, 세계기록유산 아태 지역목록 등재 서울 문화재청은 올해 6월 등재 신청한 ..."
12036,생활/문화,생활/문화 일반,극지 기본계획 수립 사전 브리핑 세종 기자 조승환 해양수산부 장관이 21일 오후...
13197,생활/문화,생활/문화 일반,제3회 제주비엔날레 참여작가 펠리시 데스티엔 도르브의 'Cosmographies' ...
9518,생활/문화,생활/문화 일반,이기봉 개인전 'Where You Stand' 서울 기자 이기봉 작가의 개인전 ...
14319,생활/문화,생활/문화 일반,"내방가사, 세계기록유산 아태 지역목록 등재 서울 문화재청은 올해 6월 등재 신청한 ..."
5609,생활/문화,날씨,'전주향교의 가을' 전주 기자 완연한 가을 날씨가 이어진 10일 전북 전주시 전...
